# SrFit example for a simple linear fit to a noisy data.

Simulate linear data with some random Gaussian noise and plot the generated "observed" data (xobs, yobs).

In [ ]:
%matplotlib inline
from matplotlib.pyplot import *
from __future__ import print_function
import numpy as np
xobs = np.arange(-10, 10.1)
dyobs = 0.3 * np.ones_like(xobs)
yobs = 0.5 * xobs + 3 + dyobs * np.random.randn(xobs.size)
plot(xobs, yobs, 'x')
title('y = 0.5*x + 3 generated with a normal noise at sigma=0.3');

We are going to define a line fitting regression using SrFit.
At first we create a SrFit Profile object that holds the observed data.

In [ ]:
from diffpy.srfit.fitbase import Profile
linedata = Profile()
linedata.setObservedProfile(xobs, yobs, dyobs)

The second step is to create a FitContribution object, which associates observed profile with a mathematical model for the dependent variable.

In [ ]:
from diffpy.srfit.fitbase import FitContribution
linefit = FitContribution('linefit')
linefit.setProfile(linedata)
linefit.setEquation("A * x + B")

 SrFit objects can be examined by calling their **show()** function.  SrFit
 parses the model equation and finds two parameters A, B at independent
 variable x.  The values of parameters A, B are at this stage undefined.


In [ ]:
linefit.show()

We can set A and B to some specific values and calculate model
observations.  The x and y attributes of the FitContribution are 
the observed values, which may be re-sampled or truncated to a shorter 
fitting range.

In [ ]:
linefit.A
linefit.A = 3
linefit.B = 5
print(linefit.A, linefit.A.value)
print(linefit.B, linefit.B.value)

`linefit.evaluate()` returns the modeled values and `linefit.residual()`,
the difference between observed and modeled data scaled by estimated
standard deviations.

In [ ]:
print("linefit.evaluate() =", linefit.evaluate())
print("linefit.residual() =", linefit.residual())
plot(xobs, yobs, 'x', linedata.x, linefit.evaluate(), '-')
title('Line simulated at A=3, B=5')

 We want to find the optimum model parameters that fit the simulated curve
 to the observations.  This is done by associating FitContribution with
 a FitRecipe object.  FitRecipe can manage multiple fit contributions and
 optimize all models to fit their respective profiles.

In [ ]:
from diffpy.srfit.fitbase import FitRecipe
rec = FitRecipe()

The `clearFitHooks()` function suppresses printout of iteration numbers.  The `addContribution()` function includes the specified FitContribution in the FitRecipe, which acts as a top-level manager of all associated fits. 

In [ ]:
rec.clearFitHooks()
rec.addContribution(linefit)
rec.show()

 FitContributions may have many parameters.  We need to tell the recipe
 which of them should be controlled and potentially optimized in a fit.

In [ ]:
rec.addVar(rec.linefit.A)
rec.addVar(rec.linefit.B)

 The call of the addVar function also created two attributes A and B for the rec object,
 which link to the A and B parameters of the linefit contribution.


In [ ]:
print("rec.A =", rec.A)
print("rec.A.value =", rec.A.value)

 The names of the declared variables are stored in the `rec.names` attribute
 and the corresponding values in `rec.values`.

In [ ]:
print("rec.values =", rec.values)
print("rec.names =", rec.names)

 Finally the recipe objects provides a residual() function to calculate
 the difference between the observed and simulated values.  The residual
 function can accept a list of new variable values in the same order as
 rec.names.

In [ ]:
print("rec.residual() =", rec.residual())
print("rec.residual([2, 4]) =", rec.residual([2, 4]))

The FitRecipe.residual function can be directly used with the scipy
leastsq function for minimizing a sum of squares.

In [ ]:
from scipy.optimize import leastsq
leastsq(rec.residual, rec.values)

 Recipe variables and the linked line-function parameters are set to the
 new optimized values.

In [ ]:
print(rec.names, "-->", rec.values)
linefit.show()

 The calculated function is available in the `ycalc` attribute of the profile.
 It can be also accessed from the `linefit` contribution attribute of the
 recipe as `rec.linefit.profile.ycalc`.

In [ ]:
plot(linedata.x, linedata.y, 'x', linedata.x, linedata.ycalc, '-')
title('Line fit using the leastsq least-squares optimizer');

The `FitRecipe.scalarResidual()` function returns the sum of squares and can
be used with a minimizer that requires a scalar function:

In [ ]:
from scipy.optimize import fmin
fmin(rec.scalarResidual, [1, 1])
print(rec.names, "-->", rec.values)
plot(linedata.x, linedata.y, 'x', linedata.x, linedata.ycalc, '-')
title('Line fit using the fmin scalar optimizer');

For a converged fit recipe, the details of the fit can be extracted
 with the FitResults class.

In [ ]:
from diffpy.srfit.fitbase import FitResults
res = FitResults(rec)
print(res)

Variables defined in the recipe can be fixed to a constant value.

In [ ]:
rec.fix(B=0)

The fixed variables can be checked using the "fixednames" and
 "fixedvalues" attributes of a recipe.

In [ ]:
print("free:", rec.names, "-->", rec.names)
print("fixed:", rec.fixednames, "-->", rec.fixedvalues)

The fit can be rerun with a constant variable B.

In [ ]:
leastsq(rec.residual, rec.values)
print(FitResults(rec))
plot(linedata.x, linedata.y, 'x', linedata.x, linedata.ycalc, '-')
title('Line fit for variable B fixed to B=0')

Fixed variables may be released with the `free()` function.
 Calling it as `free("all")` releases all fixed variables.

In [ ]:
rec.free('all')

Variables may be constrained to a result of an expression.

In [ ]:
rec.constrain(rec.A, "2 * B")

Perform linear fit where the slope value must be two times the offset.

In [ ]:
leastsq(rec.residual, rec.values)
print(FitResults(rec))
plot(linedata.x, linedata.y, 'x', linedata.x, linedata.ycalc, '-')
title('Line fit for variable A constrained to A = 2*B')

Constraint expressions can be removed by calling the unconstrain function.

In [ ]:
rec.unconstrain(rec.A)

 Variables may be restrained to a specific range.  Here "ub" is the upper
 boundary and "sig" acts as a standard deviation for ((x - ub)/sig)**2
 penalty function.

In [ ]:
arst = rec.restrain(rec.A, ub=0.2, sig=0.001)

Perform fit with the line slope restrained to a maximum value of 0.2:

In [ ]:
leastsq(rec.residual, rec.values)
print(FitResults(rec))
plot(linedata.x, linedata.y, 'x', linedata.x, linedata.ycalc, '-')
title('Line fit with A restrained to an upper bound of 0.2')